In [11]:
from dotenv import load_dotenv
import os
import openai 
import json 

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [12]:
client = openai.Client()

In [13]:
def saudacao_por_periodo(hora):
    if 5 <= 5 < 12:
        return json.dumps({"saudacao":"Bom dia!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao":"Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao":"Boa noite!"})
    else:
        return json.dumps({"saudacao":"Boa madrugada!"})

In [14]:
tools = [
    {
        "type":"function",
        "function":{
            "name":"saudacao_por_periodo",
            "description":"Retorna uma saudacao baseada na hora do dia",
            "parameters":{
                "type":"object",
                "properties":{
                    "hora":{
                        "type":"integer",
                        "description":"A hora do dia em formato de 24h"
                    }
                },
                "required":["hora"]
            }
        }
    }
]

In [15]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [22]:
mensagens = [{"role":"user","content":"Qual saudação o modelo me dá se for 22h"}]

In [23]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice="auto"
)

In [25]:
mensagem_respo = resposta.choices[0].message
mensagem_respo

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VpToSTObbfnWfGpBRwdUwXw8', function=Function(arguments='{"hora":22}', name='saudacao_por_periodo'), type='function')])

In [26]:
tools_calls = mensagem_respo.tool_calls
tools_calls

[ChatCompletionMessageToolCall(id='call_VpToSTObbfnWfGpBRwdUwXw8', function=Function(arguments='{"hora":22}', name='saudacao_por_periodo'), type='function')]

In [27]:
if tools_calls:
    mensagens.append(mensagem_respo)
    for tool_call in tools_calls:
        function_name = tool_call.function.name 
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )

        mensagens.append(
            {
                "tool_call_id":tool_call.id,
                "role":"tool",
                "name": function_name,
                "content":function_response
            }
        )
    
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens
    )


In [28]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='A saudação que o modelo me dá se for 22h é "Bom dia!".', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)